# Visualization Project

TEAMLAB Intern Researcher 서용득

# Topic

- 품목별, 품종별, 산지별 등 다양한 기준에 따른 농산물들의 거래량과 거래 가격 파악
- 시간의 흐름에 따른 농산물의 거래량과 거래 가격 변동 추이 파악

## 데이터 출처
* DACON - 2021 농산물 가격예측 AI 경진대회 (https://dacon.io/competitions/official/235801/data)
* 2016년 1월 ~ 2020년 9월의 전국 농산물 도매시장 거래정보 데이터

## 데이터 정보

기간 : 2016-01 ~ 2020-09

- train.csv 
  - date : 일자
  - 요일 : 요일
  - 품목_거래량(kg) : 해당 품목의 거래량
  - 품목_가격(원/kg) : 해당 품목의 kg당 가격
  - 품목_가격 산출 방식 : 품목 또는 품종의 총 거래금액 / 총 거래량 (취소된 거래내역 제외)  
- train_AT_TSALET_ALL : 월별 전국 도매시장 거래정보 데이터
  - SALEDATE: 경락 일자
  - WHSAL_NM: 도매시장
  - CMP_NM: 법인
  - PUM_NM: 품목
  - KIND_NM: 품종
  - DAN_NM: 단위
  - POJ_NM: 포장
  - SIZE_NM: 크기
  - LV_NM: 등급
  - SAN_NM: 산지
  - DANQ: 단위중량
  - QTY: 물량
  - COST: 단가
  - TOT_QTY: 총물량 (음수로 집계된 값은 거래 취소 내역)
  - TOT_AMT: 총금액

## Hypothesis
- **가설 1** : 김장철이 되면 배추 가격이 상승할 것이다.  

- **가설 2-1** : 귤의 주 생산지는 제주도일 것이다.
- **가설 2-2** : 제주도에서는 다른 농산물보다 귤을 가장 많이 생산할 것이다.  
    
- **가설 3** : 제주도에서 멀리 있는 지역의 도매시장에서 일어난 거래일수록 거래 가격이 비쌀 것이다.  
    
- **가설 4** : 코로나 발생(2020년 1월 기준) 이후로 중국산 농산물의 거래량은 줄었을 것이고 가격은 올랐을 것이다.  

- **가설 5** : 2016년에 비해 2019년에 열대과일의 수입량 대비 국내 생산량의 비율이 증가했을 것이다.

## 라이브러리

In [1]:
!pip install plotly

In [3]:
import os
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

from tqdm import tqdm
from datetime import datetime
from plotly.subplots import make_subplots as ms

## 설정

In [2]:
# 데이터 프레임의 모든 column을 볼 수 있도록 설정
pd.set_option('display.max_columns', None) 
# warning 제거
warnings.filterwarnings(action='ignore')

## 경로

In [4]:
DATA_DIR = "./data"
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")

## 데이터 불러오기 function

In [18]:
# 월별 도매시장 거래 DATA 불러오는 함수
def TSALET_CSV(yearmonth):
    return pd.read_csv(os.path.join(DATA_DIR, "train_AT_TSALET_ALL/AT_TSALET_ALL_{}.csv".format(yearmonth)), parse_dates = ['SALEDATE'])

# 연별 도매시장 DataFrame을 concatenate하는 함수
def TSALET_CONCAT_YEAR(year):
    dt_index = pd.date_range(start="{}0101".format(str(year)), end="{}1231".format(str(year)), freq = "M")
    dt_list = dt_index.strftime("%Y%m").tolist()
    return pd.concat([TSALET_CSV(int(i)) for i in tqdm(dt_list)])

## DATA - 품목별 거래량 합, 단가 평균 (2016-01 ~ 2020-09)

In [5]:
df = pd.read_csv(TRAIN_PATH, parse_dates=["date"], index_col = "date").iloc[:, 0:33] # 이후 5종은 품목이 아니라 품종이라 제외
df.head()

,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg)
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0
2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,5210.0,994328.1,1716.0,262615.5,1212.0,1074699.1,345.0,122613.5,9923.0,427435.1,2153.0,82113.5,3960.0,558950.2,794.0,104930.3,4871.0,277326.5,2440.0,159800.0,1750.0,291057.0,1834.0
2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,5387.0,787716.0,1715.0,221850.5,1197.0,825681.9,350.0,79055.9,9529.0,334636.8,2220.0,80144.0,3333.0,444353.7,763.0,100699.5,5129.0,218465.2,2437.0,153084.0,1822.0,194626.5,1833.0


## DATA - 전국 도매시장의 월별 거래정보

In [261]:
TSALET_CSV(201601)[200:205]

,SALEDATE,WHSAL_NM,CMP_NM,PUM_NM,KIND_NM,DAN_NM,POJ_NM,SIZE_NM,LV_NM,SAN_NM,DANQ,QTY,COST,TOT_QTY,TOT_AMT
200,2016-01-09,부산엄궁도매,부산청과,우엉,우엉(일반),kg,단,.,보통,경기도 여주군,1.0,48,1622.0,48.0,77856
201,2016-01-09,부산엄궁도매,부산청과,우엉,우엉(일반),kg,단,.,보통,경기도 여주군,1.0,494,1889.0,494.0,933166
202,2016-01-09,부산엄궁도매,부산청과,우엉,우엉(일반),kg,단,.,보통,경기도 여주군,1.0,152,1411.0,152.0,214472
203,2016-01-09,부산엄궁도매,부산청과,우엉,우엉(일반),kg,단,.,보통,경기도 여주군,1.0,472,1400.0,472.0,660800
204,2016-01-09,부산엄궁도매,부산청과,사과,미시마,kg,상자,.,보통,경상북도 청송군,10.0,1,41600.0,10.0,41600


## 가설 1. 김장철이 되면 배추 가격은 상승할 것이다.

- **코드 수정**
    - `drop()`을 최대한 사용하지 않고 코드를 짜는 것이 좋음

In [16]:
# 일요일인 것 제외하기
df = df[df["요일"] != "일요일"]
# 행 전체가 0인 행 제외하기
df["total_transaction"] = df.sum(axis = 1)
df = df[df.total_transaction != 0]
df.head()

,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),total_transaction
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,2390203.6
2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,5210.0,994328.1,1716.0,262615.5,1212.0,1074699.1,345.0,122613.5,9923.0,427435.1,2153.0,82113.5,3960.0,558950.2,794.0,104930.3,4871.0,277326.5,2440.0,159800.0,1750.0,291057.0,1834.0,39913792.0
2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,5387.0,787716.0,1715.0,221850.5,1197.0,825681.9,350.0,79055.9,9529.0,334636.8,2220.0,80144.0,3333.0,444353.7,763.0,100699.5,5129.0,218465.2,2437.0,153084.0,1822.0,194626.5,1833.0,33173807.2
2016-01-06,수요일,1045507.5,442.0,1904372.1,409.0,1860569.0,1263.0,1672.0,7041.0,91531.0,5013.0,793584.9,1620.0,199151.0,1142.0,910812.3,327.0,78532.8,8123.0,357749.8,2219.0,76832.6,3133.0,396170.4,787.0,93051.3,5120.0,187900.4,2343.0,152693.1,1519.0,178811.0,1999.0,33485764.8
2016-01-07,목요일,1039925.0,448.0,1438990.2,428.0,1868011.7,1241.0,1191.0,3908.0,349913.8,4360.0,733312.0,1545.0,200119.0,992.0,836339.0,315.0,75987.7,6793.0,323944.6,2208.0,49040.9,2259.0,241115.0,770.0,60895.4,5095.0,181058.6,2291.0,148757.3,1365.0,107346.3,1921.0,30767546.0


In [29]:
# 2016년 ~ 2020년 9월의 배추 거래정보에서 거래가 일어나지 않았거나 일요일의 데이터는 제외
df_baechu = df[["배추_거래량(kg)", "배추_가격(원/kg)"]].replace(0, np.NaN).dropna(axis = 0)
sunday = []
for i in range(df_baechu.shape[0]):
    if df_baechu.index[i].weekday() == 6:
        sunday.append(df_baechu.index[i])
        
df_baechu = df_baechu.drop(index = sunday, axis = 0)

# 2016년 ~ 2020년 9월의 주별 배추 거래량, 가격
df_baechu_vol = df_baechu['배추_거래량(kg)'].resample('W').sum()["2016":"2020"]
df_baechu_price = df_baechu['배추_가격(원/kg)'].resample('W').sum()["2016":"2020"]

# save pickle
with open("./pickles/hypo1_baechu_vol_data.pickle", "wb") as fw:
    pickle.dump(df_baechu_vol, fw)
with open("./pickles/hypo1_baechu_price_data.pickle", "wb") as fw:
    pickle.dump(df_baechu_price, fw)

- **코드 수정**
    - 읽는 사람이 편하게 이해할 수 있도록 코드를 짜는 것이 중요함
    - shape 설정시 줄바꿈을 하든지 설명을 하든지 해야함

In [18]:
# load pickle
with open("./pickles/hypo1_baechu_vol_data.pickle", "rb") as fr:
    df_baechu_vol = pickle.load(fr)
with open("./pickles/hypo1_baechu_price_data.pickle", "rb") as fr:
    df_baechu_price = pickle.load(fr)

# 8,9,11,12월 세로영역
shapes_8_9 = [{'type' : 'rect',
               'xref' : 'x',
               'yref' : 'paper',
               'x0' : '{}-08-01'.format(str(year)),
               'y0' : 0,
               'x1' : '{}-09-30'.format(str(year)),
               'y1' : 1,
               'fillcolor' : 'blue',
               'opacity' : 0.1,
               'line' : {'width' : 0}}
                for year in range(2016, 2020)]
shapes_11_12 = [{'type' : 'rect',
                 'xref' : 'x',
                 'yref' : 'paper',
                 'x0' : '{}-11-01'.format(str(year)),
                 'y0' : 0,
                 'x1' : '{}-12-31'.format(str(year)),
                 'y1' : 1,
                 'fillcolor' : 'red',
                 'opacity' : 0.1,
                 'line' : {'width' : 0}}
                 for year in range(2016, 2020)]

# 시각화
fig1 = px.line(df_baechu_vol, x = df_baechu_vol.index, y = df_baechu_vol.values, 
       title = '2016 ~ 2020년 9월의 배추 거래량(kg) 변화')
fig1.update_layout(shapes = shapes_8_9 + shapes_11_12)
fig1.show()

fig2 = px.line(df_baechu_price, x = df_baechu_price.index, y = df_baechu_price.values, 
       title = '2016 ~ 2020년 9월의 배추 가격(원/kg) 변화')
fig2.update_layout(shapes = shapes_8_9 + shapes_11_12)
fig2.show()

* 위의 그래프를 통해서,
  * 배추의 거래량은 11, 12월에 급증한다. 또한 그보다 3개월 이전인 8, 9월에도 급증하는 추세를 보인다.
  * 배추의 가격은 11, 12월에는 낮은 반면, 오히려 그보다 3개월 이전인 8, 9월에 급증하는 추세를 보인다.
* 그래서 배추의 품종별로 연간 거래량과 가격이 어떻게 변화하는지 확인해보자. (2016년을 예시로 함)

* ***여기서 2016년을 예시로 든 이유는,***
  * 본인이 프로젝트를 진행하면서 많은 양의 데이터를 한 번에 처리하는데 어려움을 겪었는데, 그에 대한 나름대로의 해결 방안이 전체 데이터 중 1개 년의 데이터로만 확인해보는 것이었다.
  * 많은 양의 데이터를 처리하는 여러 방식에 대해서는 프로젝트 진행 도중에 알게 되었는데, 시간 부족으로 인해 수정하지 못했다.
  * 추후에 이와 비슷한 일을 겪는다면, 좀 더 매끄러운 데이터 처리를 할 수 있도록 해야겠다.

In [19]:
# 2016년 도매시장 데이터
df_2016 = TSALET_CONCAT_YEAR(2016)

100%|██████████| 12/12 [01:34<00:00,  7.86s/it]


In [40]:
# 2016년 배추 거래 정보에서 COST, QTY, DANQ가 0 이하이거나 빈 칸이 있는 행은 제거
# 도매시장에서 낱개 거래가 이루어져 단가의 이상치가 된다고 판단되는 행은 제거
df_2016_baechu = df_2016[df_2016["PUM_NM"]=="배추"].replace('', np.NaN).dropna()
df_2016_baechu = df_2016_baechu.drop(df_2016_baechu[(df_2016_baechu["COST"] <= 0) | (df_2016_baechu["QTY"] <= 0) | (df_2016_baechu["DANQ"] <= 0)].index, axis = 0)
# df_2016_baechu = df_2016_baechu.drop(2976757, axis = 0)
df_2016_baechu = df_2016_baechu.drop(df_2016_baechu[df_2016_baechu["POJ_NM"]=="상자"].index, axis = 0)
df_2016_baechu = df_2016_baechu.drop(df_2016_baechu[df_2016_baechu["QTY"]==1].index, axis = 0)

# 배추의 품종별로 단가의 평균을 계산해 column을 추가
df_2016_baechu["COST_PER_DANQ"] = df_2016_baechu["TOT_AMT"] / df_2016_baechu["TOT_QTY"]
grouped_df_2016_baechu_DANQ = df_2016_baechu.groupby(["KIND_NM"])["COST_PER_DANQ"].mean().sort_values(ascending = False)

# 2016년의 배추 품종별 거래량의 합
grouped_2016_baechu_kind_TOT_QTY = pd.DataFrame(df_2016_baechu.groupby(["KIND_NM", "SALEDATE"])["TOT_QTY"].sum()).reset_index()
df_2016_baechu_kind_TOT_QTY = pd.DataFrame(columns = ["TOT_QTY", "KIND_NM"])

for kind in grouped_2016_baechu_kind_TOT_QTY["KIND_NM"].unique():
    df_temp = grouped_2016_baechu_kind_TOT_QTY[grouped_2016_baechu_kind_TOT_QTY["KIND_NM"]==kind].drop(["KIND_NM"],
                                                                                             axis = 1).set_index('SALEDATE')
    df_temp = pd.DataFrame(df_temp["TOT_QTY"].resample("W").sum()).replace(0, np.NaN).dropna()
    df_temp["KIND_NM"] = [kind for i in range(df_temp.shape[0])]
    df_2016_baechu_kind_TOT_QTY = pd.concat([df_2016_baechu_kind_TOT_QTY, df_temp])

df_2016_baechu_kind_TOT_QTY = df_2016_baechu_kind_TOT_QTY.reset_index().rename(columns = {"index":"SALEDATE"})

# 2016년 배추 품종별 단가의 평균
grouped_2016_baechu_kind_DANQ = pd.DataFrame(df_2016_baechu.groupby(["KIND_NM", "SALEDATE"])["COST_PER_DANQ"].mean()).reset_index()
df_2016_baechu_kind_DANQ = pd.DataFrame(columns = ["COST_PER_DANQ", "KIND_NM"])

for kind in grouped_2016_baechu_kind_DANQ["KIND_NM"].unique():
    df_temp = grouped_2016_baechu_kind_DANQ[grouped_2016_baechu_kind_DANQ["KIND_NM"]==kind].drop(["KIND_NM"],
                                                                                             axis = 1).set_index('SALEDATE')
    df_temp = pd.DataFrame(df_temp["COST_PER_DANQ"].resample("W").mean()).replace(0, np.NaN).dropna()
    df_temp["KIND_NM"] = [kind for i in range(df_temp.shape[0])]
    df_2016_baechu_kind_DANQ = pd.concat([df_2016_baechu_kind_DANQ, df_temp])
    
df_2016_baechu_kind_DANQ = df_2016_baechu_kind_DANQ.reset_index().rename(columns = {"index":"SALEDATE"})

# save pickle
with open("./pickles/hypo1_baechu_kind_TOT_QTY_data.pickle", "wb") as fw:
    pickle.dump(df_2016_baechu_kind_TOT_QTY, fw)
with open("./pickles/hypo1_baechu_kind_DANQ_data.pickle", "wb") as fw:
    pickle.dump(df_2016_baechu_kind_DANQ, fw)

In [11]:
# load pickle
with open("./pickles/hypo1_baechu_kind_TOT_QTY_data.pickle", "rb") as fr:
    df_2016_baechu_kind_TOT_QTY = pickle.load(fr)
with open("./pickles/hypo1_baechu_kind_DANQ_data.pickle", "rb") as fr:
    df_2016_baechu_kind_DANQ = pickle.load(fr)

# 시각화
fig1 = px.line(df_2016_baechu_kind_TOT_QTY, x = 'SALEDATE', y = 'TOT_QTY', color = "KIND_NM",
              title = "2016년 배추 품종별 거래량(kg) 변화")
fig1.update_layout(
    shapes = [{'type' : 'rect','xref' : 'x','yref' : 'paper','x0' : '2016-08-01','y0' : 0,'x1' : '2016-09-30',
                'y1' : 1,'fillcolor' : 'blue','opacity' : 0.1,'line' : {'width' : 0}},
              {'type' : 'rect','xref' : 'x','yref' : 'paper','x0' : '2016-11-01','y0' : 0,'x1' : '2016-12-31',
                'y1' : 1,'fillcolor' : 'red','opacity' : 0.1,'line' : {'width' : 0}}],
      width = 1000, height = 600)
fig1.show()

fig2 = px.line(df_2016_baechu_kind_DANQ, x = 'SALEDATE', y = 'COST_PER_DANQ', color = "KIND_NM",
              title = "2016년 배추 품종별 가격(원/kg) 변화")
fig2.update_layout(
    shapes = [{'type' : 'rect','xref' : 'x','yref' : 'paper','x0' : '2016-08-01','y0' : 0,'x1' : '2016-09-30',
                'y1' : 1,'fillcolor' : 'blue','opacity' : 0.1,'line' : {'width' : 0}},
              {'type' : 'rect','xref' : 'x','yref' : 'paper','x0' : '2016-11-01','y0' : 0,'x1' : '2016-12-31',
                'y1' : 1,'fillcolor' : 'red','opacity' : 0.1,'line' : {'width' : 0}}],
    width = 1000, height = 600)
fig2.show()

* **위 두 그래프에서,**
  * 거래량은 8, 9월에 고냉지배추가 증가했고, 11, 12월에는 김장(가을)배추가 크게 증가하는 추세를 보인다.
  * 가격은 우거지와 절임배추를 제외하면, 대부분 품종의 배추들이 8,9월에 증가하는 추세를 보인다.
* 김장철인 11, 12월에 김장배추의 거래량이 증가한다는 사실은 알 수 있지만,
* 가격은 전체적으로 8, 9월을 중심으로 증가하며, 김장철에는 크게 영향을 받지 않는다는 사실을 알 수 있다.

## 가설 2-1. 귤의 주 생산지는 제주도일 것이다.

In [ ]:
# 2016년 도매시장 데이터
df_2016 = TSALET_CONCAT_YEAR(2016)

In [256]:
# 데이터 전처리
df_2016_gamgul = df_2016[df_2016["PUM_NM"]=="감귤"]
df_2016_gamgul["SAN_NM"].replace('', np.NaN, inplace = True)
df_2016_gamgul = df_2016_gamgul.dropna(subset = ["SAN_NM"], axis = 0)
df_2016_gamgul = df_2016_gamgul.drop((df_2016_gamgul[df_2016_gamgul['COST'] <= 0]).index, axis=0)
df_2016_gamgul = df_2016_gamgul.drop((df_2016_gamgul[df_2016_gamgul['QTY'] <= 0]).index, axis=0)
df_2016_gamgul = df_2016_gamgul.drop((df_2016_gamgul[df_2016_gamgul['DANQ'] <= 0]).index, axis=0)
df_2016_gamgul = df_2016_gamgul.replace("제주 서귀포시", "제주도 서귀포시")
df_2016_gamgul = df_2016_gamgul.replace("제주 제주시", "제주도 제주시")

# 2016년도 전국 감귤 생산지별 거래량 합
df_2016_gamgul_san = df_2016_gamgul.groupby(["SAN_NM"])["TOT_QTY"].sum().sort_values(ascending = False)

# 제주도를 제외한 생산지는 기타로 묶음
df_2016_gamgul_san_etc_name = '기타(150여 개 지역)'
df_2016_gamgul_san_etc_value = df_2016_gamgul_san[5:].sum()

# 2016년도 전국 감귤 생산지별 거래량 합에 제주도를 제외한 생산지는 모두 묶어 기타 row로 처리
df_2016_gamgul_san = df_2016_gamgul_san[0:5]
df_2016_gamgul_san.loc[df_2016_gamgul_san_etc_name] = df_2016_gamgul_san_etc_value

# save pickle
with open("./pickles/hypo2_2016_gamgul_san_data.pickle", "wb") as fw:
    pickle.dump(df_2016_gamgul_san, fw)

In [20]:
# load pickle
with open("./pickles/hypo2_2016_gamgul_san_data.pickle", "rb") as fr:
    df_2016_gamgul_san = pickle.load(fr)
    
# 시각화 - 2016년도 전국 감귤 생산지별 거래량 합
colors = ["orange",] * 6
colors[5] = 'lightslategray'

fig = go.Figure(data = [go.Bar(x = df_2016_gamgul_san.index, y = df_2016_gamgul_san.values,
                               marker_color = colors)])
fig.update_layout(title_text = '2016년도 전국 감귤 생산지별 거래량 합(kg)', width = 1000, height = 600)

fig.show()

* **위 그래프에서,**
  * 감귤 생산량에 있어 전국의 기타 150여 개 지역이 차지하는 양에 비해 제주도가 차지하는 양이 압도적으로 많았다.
* 따라서, 감귤의 최대 생산지는 제주도라고 할 수 있다.

## 가설 2-2. 제주도에서는 다른 농산물보다 귤을 가장 많이 생산할 것이다.

In [212]:
# 데이터 전처리
df_2016["SAN_NM"].replace('', np.NaN, inplace = True)
df_2016 = df_2016.dropna(subset = ["SAN_NM"], axis = 0)

# 2016년 제주도산 농산물의 품목별 전체 거래중량
df_2016_san_jeju = df_2016[df_2016["SAN_NM"].isin([i for i in df_2016["SAN_NM"].unique() if "제주" in i])]
grouped_san_jeju = df_2016_san_jeju.groupby(["PUM_NM"])["TOT_QTY"].sum().sort_values()
grouped_san_jeju = pd.DataFrame(data = grouped_san_jeju)

# 합이 1이 되도록 정규화
grouped_san_jeju["TOT_QTY"] /= grouped_san_jeju["TOT_QTY"].sum()

# save pickle
with open("./pickles/hypo2_piechart_data.pickle", "wb") as fw:
    pickle.dump(grouped_san_jeju, fw)

## 가설 3 : 제주도에서 멀리 있는 지역의 도매시장에서 일어난 거래일수록 거래 가격이 비쌀 것이다.

In [22]:
# load pickle
with open("./pickles/hypo2_piechart_data.pickle", "rb") as fr:
    grouped_san_jeju = pickle.load(fr)

grouped_san_jeju = grouped_san_jeju.sort_values(by = "TOT_QTY", ascending = False).head(10)

# 시각화 - 제주도가 주로 생산하는 농산물에 대한 비율
fig = px.pie(grouped_san_jeju, values = grouped_san_jeju["TOT_QTY"], names = grouped_san_jeju.index,
              color_discrete_sequence=px.colors.sequential.RdBu, title = "2016년도 제주도산 농산물 품목 비율(%) - 상위 10개 품목")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(width = 800, height = 800)

fig.show()

* 2016년도의 제주도산 농산물 중 가장 많은 비율을 차지하는 품목은 40.9%로 감귤이었다. 
* 감귤, 무, 양배추가 2016년 제주도산 농산물 중 약 80%를 차지한다.
  * 따라서 이후 가설 3에서 감귤, 무, 양배추에 대해서 거래지역별 거래 가격의 차이에 대해서 알아보기로 한다.

## 가설 3 : 제주도에서 멀리 있는 지역의 도매시장에서 일어난 거래일수록 거래 가격이 비쌀 것이다.

In [214]:
# 2016년 제주도산 감귤 중 DAMQ, COST, QTY가 0이거나 음수인 행 제거
df_2016_san_jeju_gamgul = df_2016_san_jeju[df_2016_san_jeju["PUM_NM"]=="감귤"]
df_2016_san_jeju_gamgul = df_2016_san_jeju_gamgul.drop((df_2016_san_jeju_gamgul[df_2016_san_jeju_gamgul['COST'] <= 0]).index, axis=0)
df_2016_san_jeju_gamgul = df_2016_san_jeju_gamgul.drop((df_2016_san_jeju_gamgul[df_2016_san_jeju_gamgul['QTY'] <= 0]).index, axis=0)
df_2016_san_jeju_gamgul = df_2016_san_jeju_gamgul.drop((df_2016_san_jeju_gamgul[df_2016_san_jeju_gamgul['DANQ'] <= 0]).index, axis=0)


# 2016년 제주도산 감귤의 단위가격 평균 
df_2016_san_jeju_gamgul['COST_PER_DANQ'] = df_2016_san_jeju_gamgul["TOT_AMT"] / df_2016_san_jeju_gamgul["TOT_QTY"]
grouped_2016_san_jeju_gamgul_WHSAL = pd.DataFrame(df_2016_san_jeju_gamgul.groupby(["WHSAL_NM"])["COST_PER_DANQ"].mean()).reset_index()

# 도매시장의 좌표
latitude = [37.73613, 35.18385, 35.11620, 37.61263, 36.15712, 35.90339, 36.36528, 36.35779, 35.21407, 35.12858,
            37.49403, 37.55287, 37.25562, 34.91683, 36.58981, 37.30957, 37.38298, 35.53989, 37.35449, 35.91878, 
            37.44229, 37.52219, 35.86807, 35.58939, 35.21068, 35.25519, 35.22901, 36.86107, 36.64783, 37.91020, 
            37.00551, 36.08150]
longitude = [128.91824, 126.93501, 126.85632, 127.14465, 128.35023, 128.54235, 127.32074, 127.40658, 129.12180, 128.96510,
             127.11482, 126.82083, 127.03111, 127.53796, 128.60896, 126.85733, 126.97009, 129.34266, 127.92328, 126.91714,
             126.70494, 126.74920, 127.11701, 126.84576, 128.12295, 128.51132, 128.63746, 127.14993, 127.46439, 127.72691,
             127.92131, 129.33650]

# 도매시장의 좌표와 제주도와의 거리 column 추가
jeju = [33.52088, 126.53885]

grouped_2016_san_jeju_gamgul_WHSAL = grouped_2016_san_jeju_gamgul_WHSAL.assign(latitude = latitude, longitude = longitude)
grouped_2016_san_jeju_gamgul_WHSAL = grouped_2016_san_jeju_gamgul_WHSAL.assign(
    distance = [((grouped_2016_san_jeju_gamgul_WHSAL.iloc[i, 2] - jeju[0])**2 + 
                 (grouped_2016_san_jeju_gamgul_WHSAL.iloc[i, 3] - jeju[1])**2)**0.5 for i in range(0, 32)])

######################################################################

# 2016년 제주도산 무 중 DAMQ, COST, QTY가 0이거나 음수인 행 제거
df_2016_san_jeju_radish = df_2016_san_jeju[df_2016_san_jeju["PUM_NM"]=="무"]
df_2016_san_jeju_radish = df_2016_san_jeju_radish.drop((df_2016_san_jeju_radish[df_2016_san_jeju_radish['COST'] <= 0]).index, axis=0)
df_2016_san_jeju_radish = df_2016_san_jeju_radish.drop((df_2016_san_jeju_radish[df_2016_san_jeju_radish['QTY'] <= 0]).index, axis=0)
df_2016_san_jeju_radish = df_2016_san_jeju_radish.drop((df_2016_san_jeju_radish[df_2016_san_jeju_radish['DANQ'] <= 0]).index, axis=0)


# 2016년 제주도산 무의 단위가격 평균 확인
df_2016_san_jeju_radish['COST_PER_DANQ'] = df_2016_san_jeju_radish["TOT_AMT"] / df_2016_san_jeju_radish["TOT_QTY"]
grouped_2016_san_jeju_radish_WHSAL = pd.DataFrame(df_2016_san_jeju_radish.groupby(["WHSAL_NM"])["COST_PER_DANQ"].mean()).reset_index()

# 도매시장의 좌표와 제주도와의 거리 column 추가
latitude2 = latitude[0:14] + latitude[15:29] + latitude[30:]
longitude2 = longitude[0:14] + longitude[15:29] + longitude[30:]
grouped_2016_san_jeju_radish_WHSAL = grouped_2016_san_jeju_radish_WHSAL.assign(latitude = latitude2,longitude = longitude2)
grouped_2016_san_jeju_radish_WHSAL = grouped_2016_san_jeju_radish_WHSAL.assign(
    distance = [((grouped_2016_san_jeju_radish_WHSAL.iloc[i, 2] - jeju[0])**2 + 
                 (grouped_2016_san_jeju_radish_WHSAL.iloc[i, 3] - jeju[1])**2)**0.5 for i in range(0, 30)])

######################################################################

# 2016년 제주도산 양배추 중 DAMQ, COST, QTY가 0이거나 음수인 행 제거
df_2016_san_jeju_cabbage = df_2016_san_jeju[df_2016_san_jeju["PUM_NM"]=="양배추"]
df_2016_san_jeju_cabbage = df_2016_san_jeju_cabbage.drop((df_2016_san_jeju_cabbage[df_2016_san_jeju_cabbage['COST'] <= 0]).index, axis=0)
df_2016_san_jeju_cabbage = df_2016_san_jeju_cabbage.drop((df_2016_san_jeju_cabbage[df_2016_san_jeju_cabbage['QTY'] <= 0]).index, axis=0)
df_2016_san_jeju_cabbage = df_2016_san_jeju_cabbage.drop((df_2016_san_jeju_cabbage[df_2016_san_jeju_cabbage['DANQ'] <= 0]).index, axis=0)


# 2016년 제주도산 양배추의 단위가격 평균 
df_2016_san_jeju_cabbage['COST_PER_DANQ'] = df_2016_san_jeju_cabbage["TOT_AMT"] / df_2016_san_jeju_cabbage["TOT_QTY"]
grouped_2016_san_jeju_cabbage_WHSAL = pd.DataFrame(df_2016_san_jeju_cabbage.groupby(["WHSAL_NM"])["COST_PER_DANQ"].mean()).reset_index()


# 도매시장의 좌표와 제주도와의 거리 column 추가
grouped_2016_san_jeju_cabbage_WHSAL = grouped_2016_san_jeju_cabbage_WHSAL.assign(latitude = latitude2,longitude = longitude2)
grouped_2016_san_jeju_cabbage_WHSAL = grouped_2016_san_jeju_cabbage_WHSAL.assign(
    distance = [((grouped_2016_san_jeju_cabbage_WHSAL.iloc[i, 2] - jeju[0])**2 + 
                 (grouped_2016_san_jeju_cabbage_WHSAL.iloc[i, 3] - jeju[1])**2)**0.5 for i in range(0, 30)])

# save pickle
with open("./pickles/hypo3_map_gamgul_data.pickle", "wb") as fw:
    pickle.dump(grouped_2016_san_jeju_gamgul_WHSAL, fw)
with open("./pickles/hypo3_map_radish_data.pickle", "wb") as fw:
    pickle.dump(grouped_2016_san_jeju_radish_WHSAL, fw)
with open("./pickles/hypo3_map_cabbage_data.pickle", "wb") as fw:
    pickle.dump(grouped_2016_san_jeju_cabbage_WHSAL, fw)

In [24]:
# load pickle
with open("./pickles/hypo3_map_gamgul_data.pickle", "rb") as fr:
    grouped_2016_san_jeju_gamgul_WHSAL = pickle.load(fr)
with open("./pickles/hypo3_map_radish_data.pickle", "rb") as fr:
    grouped_2016_san_jeju_radish_WHSAL = pickle.load(fr)
with open("./pickles/hypo3_map_cabbage_data.pickle", "rb") as fr:
    grouped_2016_san_jeju_cabbage_WHSAL = pickle.load(fr)
    
# 지도 시각화
px.set_mapbox_access_token(open("mapbox_token.py").read())

# 감귤
fig1 = px.scatter_mapbox(grouped_2016_san_jeju_gamgul_WHSAL, lat = 'latitude', lon = 'longitude', size = 'distance',
                        color = 'COST_PER_DANQ', hover_name = 'WHSAL_NM',
                        color_continuous_scale = px.colors.sequential.Oranges)
fig1.add_trace(go.Scattermapbox(
    mode = "markers",
    name = "제주",
    lon = [126.53885],
    lat = [33.52088],
    marker = {'size': 15, 'color': "yellow"}))
fig1.update_layout(mapbox_style='carto-darkmatter', mapbox_zoom=6, showlegend = False, 
                  title_text = "2016년 제주도산 감귤의 거래지역별 단가(원/kg)", title_x=0.5, width = 1000, height = 750, mapbox_center = {"lat": 35.8, "lon": 127.5})

# 무
fig2 = px.scatter_mapbox(grouped_2016_san_jeju_radish_WHSAL, lat = 'latitude', lon = 'longitude', size = 'distance',
                        color = 'COST_PER_DANQ', hover_name = 'WHSAL_NM',
                        color_continuous_scale = px.colors.sequential.Reds)
fig2.add_trace(go.Scattermapbox(
    mode = "markers",
    name = "제주",
    lon = [126.53885],
    lat = [33.52088],
    marker = {'size': 15, 'color': "yellow"}))
fig2.update_layout(mapbox_style='carto-darkmatter', mapbox_zoom=6, showlegend = False, 
                  title_text = "2016년 제주도산 무의 거래지역별 단가(원/kg)", title_x=0.5, width = 1000, height = 750, mapbox_center = {"lat": 35.8, "lon": 127.5})

# 양배추
fig3 = px.scatter_mapbox(grouped_2016_san_jeju_cabbage_WHSAL, lat = 'latitude', lon = 'longitude', size = 'distance',
                        color = 'COST_PER_DANQ', hover_name = 'WHSAL_NM',
                        color_continuous_scale = px.colors.sequential.Greens)
fig3.add_trace(go.Scattermapbox(
    mode = "markers",
    name = "제주",
    lon = [126.53885],
    lat = [33.52088],
    marker = {'size': 15, 'color': "yellow"}))
fig3.update_layout(mapbox_style='carto-darkmatter', mapbox_zoom=6, showlegend = False, 
                  title_text = "2016년 제주도산 양배추의 거래지역별 단가(원/kg)", title_x=0.5, width = 1000, height = 750, mapbox_center = {"lat": 35.8, "lon": 127.5})

fig1.show()
fig2.show()
fig3.show()

* 위 지도에서 원의 크기는 제주도와의 거리를, 색은 단가를 의미한다.
* 제주도산 감귤, 무, 양배추의 거래지역별 단가에 대해서 알아봤는데,
  * **제주도와의 거리와 단가 사이에는 눈에 띄는 관계가 발견되지는 않았다.**

* **시각화를 해 본 후**
  * 분석 시 지도상 일직선 거리를 이용해 알아보고자 했기 때문에 뚜렷한 관계가 나타나지 않은 것 같다.
  * 제주도와의 거리보다는 지역별로 존재하는 구매방식이나 유통구조의 차이와 같은 다른 요인에 의해 단가의 차이가 나타날 수도 있을 것 같다는 생각을 했다.
  * 거래지역별로 단가가 차이가 나는 이유에 대해서 제대로 알아보고자 한다면, 추가적인 정보가 필요할 것 같다.

In [ ]:
# 2016년도 제주산 감귤의 가격에 대한 상관관계분석
grouped_2016_san_jeju_gamgul_WHSAL = pd.DataFrame(df_2016_san_jeju_gamgul.groupby(["WHSAL_NM"])["COST_PER_DANQ", "TOT_QTY"].mean()).reset_index()
grouped_2016_san_jeju_gamgul_WHSAL = grouped_2016_san_jeju_gamgul_WHSAL.assign(latitude = latitude, longitude = longitude)
grouped_2016_san_jeju_gamgul_WHSAL = grouped_2016_san_jeju_gamgul_WHSAL.assign(
    distance = [((grouped_2016_san_jeju_gamgul_WHSAL.iloc[i, 3] - jeju[0])**2 + 
                 (grouped_2016_san_jeju_gamgul_WHSAL.iloc[i, 4] - jeju[1])**2)**0.5 for i in range(0, 32)])
corr_gamgul = grouped_2016_san_jeju_gamgul_WHSAL[["COST_PER_DANQ", "TOT_QTY", "distance"]].corr()

# 2016년도 제주산 무의 가격에 대한 상관관계분석
grouped_2016_san_jeju_radish_WHSAL = pd.DataFrame(df_2016_san_jeju_radish.groupby(["WHSAL_NM"])["COST_PER_DANQ", "TOT_QTY"].mean()).reset_index()
grouped_2016_san_jeju_radish_WHSAL = grouped_2016_san_jeju_radish_WHSAL.assign(latitude = latitude2, longitude = longitude2)
grouped_2016_san_jeju_radish_WHSAL = grouped_2016_san_jeju_radish_WHSAL.assign(
    distance = [((grouped_2016_san_jeju_radish_WHSAL.iloc[i, 3] - jeju[0])**2 + 
                 (grouped_2016_san_jeju_radish_WHSAL.iloc[i, 4] - jeju[1])**2)**0.5 for i in range(0, 30)])
corr_radish = grouped_2016_san_jeju_radish_WHSAL[["COST_PER_DANQ", "TOT_QTY", "distance"]].corr()

# 2016년도 제주산 양배추의 가격에 대한 상관관계분석
grouped_2016_san_jeju_cabbage_WHSAL = pd.DataFrame(df_2016_san_jeju_cabbage.groupby(["WHSAL_NM"])["COST_PER_DANQ", "TOT_QTY"].mean()).reset_index()
grouped_2016_san_jeju_cabbage_WHSAL = grouped_2016_san_jeju_cabbage_WHSAL.assign(latitude = latitude2, longitude = longitude2)
grouped_2016_san_jeju_cabbage_WHSAL = grouped_2016_san_jeju_cabbage_WHSAL.assign(
    distance = [((grouped_2016_san_jeju_cabbage_WHSAL.iloc[i, 3] - jeju[0])**2 + 
                 (grouped_2016_san_jeju_cabbage_WHSAL.iloc[i, 4] - jeju[1])**2)**0.5 for i in range(0, 30)])
corr_cabbage = grouped_2016_san_jeju_cabbage_WHSAL[["COST_PER_DANQ", "TOT_QTY", "distance"]].corr()


# save pickle
with open("./pickles/hypo3_map_gamgul_corr_data.pickle", "wb") as fw:
    pickle.dump(corr_gamgul, fw)
with open("./pickles/hypo3_map_radish_corr_data.pickle", "wb") as fw:
    pickle.dump(corr_radish, fw)
with open("./pickles/hypo3_map_cabbage_corr_data.pickle", "wb") as fw:
    pickle.dump(corr_cabbage, fw)

In [31]:
# load pickle
with open("./pickles/hypo3_map_gamgul_corr_data.pickle", "rb") as fr:
    corr_gamgul = pickle.load(fr)
with open("./pickles/hypo3_map_radish_corr_data.pickle", "rb") as fr:
    corr_radish = pickle.load(fr)
with open("./pickles/hypo3_map_cabbage_corr_data.pickle", "rb") as fr:
    corr_cabbage = pickle.load(fr)
    
# heatmap 시각화
fig1 = px.imshow(corr_gamgul,
                color_continuous_scale = px.colors.diverging.RdBu, text_auto=True,
                color_continuous_midpoint = 0, width = 500, height = 500,
                title = "2016년도 제주산 감귤의 가격에 대한 상관관계분석")
fig1.show()

fig2 = px.imshow(corr_radish,
                color_continuous_scale = px.colors.diverging.RdBu, text_auto=True,
                color_continuous_midpoint = 0, width = 500, height = 500,
                title = "2016년도 제주산 무의 가격에 대한 상관관계분석")
fig2.show()

fig3 = px.imshow(corr_cabbage,
                color_continuous_scale = px.colors.diverging.RdBu, text_auto=True,
                color_continuous_midpoint = 0, width = 500, height = 500,
                title = "2016년도 제주산 양배추의 가격에 대한 상관관계분석")
fig3.show()



* **추가적으로 단가, 거래중량, 거래지역과의 거리에 대한 상관관계분석을 진행**
  * 유의미한 관계를 찾지 못했다.
  * 산지로부터 거래지역과의 거리 이외에 나타나지 않은 요인에 의해 거래 가격이 변화하는 것으로 생각할 수 있다.
  * 거래 가격이 변화하게끔 하는 요인이 어떤 것이 있는지에 대해서는 더 구체적인 데이터 분석을 통해 알 수 있을 것이라고 판단된다.

## 가설 4 : 코로나 발생(2020년 1월 기준) 이후로 중국산 농산물의 거래량은 줄었을 것이고 가격은 올랐을 것이다.

In [ ]:
df_2019 = TSALET_CONCAT_YEAR(2019)

In [ ]:
dt_index = pd.date_range(start="20200101", end="20200930", freq = "M")
dt_list = dt_index.strftime("%Y%m").tolist()
df_2020 = pd.concat([TSALET_CSV(int(i)) for i in tqdm(dt_list)])

In [148]:
# 2019년 ~ 2020년의 중국산 농산물에 대한 가격과 거래량 확인
df_2019_china = df_2019[df_2019["SAN_NM"]=="중국"]
df_2019_china = df_2019_china.drop(df_2019_china[(df_2019_china["COST"] <= 0) | (df_2019_china["DANQ"] <= 0) |
                                                 (df_2019_china["QTY"] <= 0)].index, axis = 0)
df_2019_china.groupby(["SALEDATE"])["TOT_QTY"].sum()

df_2020_china = df_2020[df_2020["SAN_NM"] == "중국"]
df_2020_china = df_2020_china.drop(df_2020_china[(df_2020_china["COST"] <= 0) | (df_2020_china["DANQ"] <= 0) |
                                                 (df_2020_china["QTY"] <= 0)].index, axis = 0)

df_19_20_china = pd.concat([df_2019_china, df_2020_china])

# 2019년 ~ 2020년 중국산 농산물의 단가 column 추가
df_19_20_china["COST_PER_DANQ"] = df_19_20_china["TOT_AMT"] / df_19_20_china["TOT_QTY"]

# 2019년 ~ 2020년 전체 농산물 거래량의 합
df_2019_2020_TOT_QTY = pd.concat([df_2019.groupby(["SALEDATE"])["TOT_QTY"].sum(),
                                  df_2020.groupby(["SALEDATE"])["TOT_QTY"].sum()])

# 2019년 전체 농산물 단가 평균
df_2019_SD_TQ_TA = df_2019[["SALEDATE", "TOT_QTY", "TOT_AMT"]]
df_2019_SD_TQ_TA = df_2019_SD_TQ_TA.replace(0, np.NaN).dropna(axis = 0)
df_2019_SD_TQ_TA["COST_PER_DANQ"] = df_2019_SD_TQ_TA["TOT_AMT"] / df_2019_SD_TQ_TA["TOT_QTY"]
grouped_2019_all_DANQ = df_2019_SD_TQ_TA.groupby(["SALEDATE"])["COST_PER_DANQ"].mean()

# 2020년 전체 농산물 단가 평균
df_2020_SD_TQ_TA = df_2020[["SALEDATE", "TOT_QTY", "TOT_AMT"]]
df_2020_SD_TQ_TA = df_2020_SD_TQ_TA.replace(0, np.NaN).dropna(axis = 0)
df_2020_SD_TQ_TA["COST_PER_DANQ"] = df_2020_SD_TQ_TA["TOT_AMT"] / df_2020_SD_TQ_TA["TOT_QTY"]
grouped_2020_all_DANQ = df_2020_SD_TQ_TA.groupby(["SALEDATE"])["COST_PER_DANQ"].mean()

# 2019년 ~ 2020년 전체 농산물 단가 평균
grouped_19_20_all_DANQ = pd.concat([grouped_2019_all_DANQ, grouped_2020_all_DANQ])

# 2019년 ~ 2020년 (전체 / 중국산) 농산물 거래량 합 변화
grouped_19_20_TOT_QTY_all = pd.DataFrame(df_2019_2020_TOT_QTY)["TOT_QTY"].resample("2W").sum()
grouped_19_20_TOT_QTY_china = pd.DataFrame(df_19_20_china.groupby(["SALEDATE"])["TOT_QTY"].sum())["TOT_QTY"].resample("2W").sum()

# 2019년 ~ 2020년 (전체 / 중국산) 농산물 단가 평균 변화
grouped_19_20_DANQ_all = pd.DataFrame(grouped_19_20_all_DANQ)["COST_PER_DANQ"].resample("2W").mean()
grouped_19_20_DANQ_china = pd.DataFrame(df_19_20_china.groupby(["SALEDATE"])["COST_PER_DANQ"].mean())["COST_PER_DANQ"].resample("2W").mean()

# save pickle
with open("./pickles/hypo4_grouped_1920_TOT_QTY_all_data.pickle", "wb") as fw:
    pickle.dump(grouped_19_20_TOT_QTY_all, fw)
with open("./pickles/hypo4_grouped_1920_TOT_QTY_china_data.pickle", "wb") as fw:
    pickle.dump(grouped_19_20_TOT_QTY_china, fw)
with open("./pickles/hypo4_grouped_1920_DANQ_all_data.pickle", "wb") as fw:
    pickle.dump(grouped_19_20_DANQ_all, fw)
with open("./pickles/hypo4_grouped_1920_DANQ_china_data.pickle", "wb") as fw:
    pickle.dump(grouped_19_20_DANQ_china, fw)

In [33]:
# load pickle
with open("./pickles/hypo4_grouped_1920_TOT_QTY_all_data.pickle", "rb") as fr:
    grouped_19_20_TOT_QTY_all = pickle.load(fr)
with open("./pickles/hypo4_grouped_1920_TOT_QTY_china_data.pickle", "rb") as fr:
    grouped_19_20_TOT_QTY_china = pickle.load(fr)
with open("./pickles/hypo4_grouped_1920_DANQ_all_data.pickle", "rb") as fr:
    grouped_19_20_DANQ_all = pickle.load(fr)
with open("./pickles/hypo4_grouped_1920_DANQ_china_data.pickle", "rb") as fr:
    grouped_19_20_DANQ_china = pickle.load(fr)

# 시각화 - 2019년 ~ 2020년 (전체 / 중국산) 농산물 거래량 합 변화
fig1= ms(specs=[[{'secondary_y': True}]])
fig1.add_trace(go.Scatter(x = grouped_19_20_TOT_QTY_all.index, y = grouped_19_20_TOT_QTY_all.values[:-1], name = "All"),
             secondary_y = False)
fig1.add_trace(go.Scatter(x = grouped_19_20_TOT_QTY_china.index, y = grouped_19_20_TOT_QTY_china.values[:-1], name = "China"),
             secondary_y = True)

fig1.add_shape(type = 'rect', x0 = '2020-01-06', y0 = 0, x1 = '2020-10-04', y1 =1, 
              fillcolor = 'green',opacity = 0.1, line = {'width' : 0}, xref='x',yref='paper')
fig1.add_shape(type = 'line', x0 = '2020-01-06', y0 = 0, x1 = '2020-01-06', y1 =1, xref='x', yref='paper',
              line = {'color' : 'green'})

fig1.update_layout(title = '2019년 ~ 2020년 (전체 / 중국산) 농산물 거래량 합 변화')

fig2= ms(specs=[[{'secondary_y': True}]])
fig2.add_trace(go.Scatter(x = grouped_19_20_DANQ_all.index, y = grouped_19_20_DANQ_all.values[:-1], name = "All"),
             secondary_y = False)
fig2.add_trace(go.Scatter(x = grouped_19_20_DANQ_china.index, y = grouped_19_20_DANQ_china.values[:-1], name = "China"),
             secondary_y = True)

fig2.add_shape(type = 'rect', x0 = '2020-01-06', y0 = 0, x1 = '2020-10-04', y1 =1, 
              fillcolor = 'green',opacity = 0.1, line = {'width' : 0}, xref='x',yref='paper')
fig2.add_shape(type = 'line', x0 = '2020-01-06', y0 = 0, x1 = '2020-01-06', y1 =1, xref='x', yref='paper',
              line = {'color' : 'green'})

fig2.update_layout(title = '2019년 ~ 2020년 (전체 / 중국산) 농산물 단가 평균 변화')

fig1.show()
fig2.show()

* 코로나 바이러스가 전파되기 시작한 시기인 2020년 1월 초를 기준으로 전후 변화를 알아보기 위해 2019년과 2020년 데이터를 활용하였다.

* **중국산 농산물의 거래량은 2020년 1월을 기준으로 하여 전과 다르게 큰 변화가 생겼다고 할 수 없다고 판단된다.**
  * 2020년 1월 중순부터 3월 초까지 거래량이 한달 반의 기간동안 감소하긴 하지만, 그 후 비슷한 폭으로 증감을 반복하는 추세를 보이기 때문에 코로나 바이러스 발생을 기준으로 거래량에 큰 변화가 생겼다고 할 수 없다고 판단된다.
* **중국산 농산물의 단가 또한 2020년 1월을 기준으로 하여 전과 다르게 큰 변화가 생겼다고 할 수 없다.**
  * 2020년 1월 초부터 2월 중순까지 단가가 한달 반의 기간동안 증가하긴 하지만, 그 후 비슷한 폭으로 증감을 반복하는 추세를 보이기 때문에 코로나 바이러스 발생을 기준으로 단가에 큰 변화가 생겼다고 할 수 없다고 판단된다.
  

* **시각화를 해 본 후**
  * 그래프를 육안으로 관찰해 볼 때에는 거래량과 가격이 2020년 1월을 기준으로 크게 변화가 나타나지 않는다고 판단이 되는데, 이 판단이 통계적으로 유의한지, 즉 통계적으로 변화가 일어났는지의 유무에 대해서는 알아보지 못했다.
  * 이는 추가적으로 진행해야 할 필요가 있어 보인다.

## 가설 5. 2016년에 비해 2019년에 열대과일의 수입량 대비 국내 생산량의 비율이 증가했을 것이다.

In [ ]:
df_2016 = TSALET_CONCAT_YEAR(2016)

In [ ]:
df_2019 = TSALET_CONCAT_YEAR(2019)

In [14]:
# 국내 생산 중인 열대과일만을 선별
tropical_fruits = ['파파야', '바나나', '용과', '파인애플', '참다래(키위)', '자몽', '람부탄']

# 2016년 열대과일의 생산지 추출
trop_2016_san_TOT_QTY = df_2016[df_2016["PUM_NM"].isin(tropical_fruits)]
trop_2016_san_TOT_QTY = pd.DataFrame(trop_2016_san_TOT_QTY.groupby(["PUM_NM", "SAN_NM"])["TOT_QTY"].sum()).reset_index()

# 2016년 열대과일의 생산지를 국내와 수입으로 분류
for i in range(560):
    if trop_2016_san_TOT_QTY.iloc[i,1][-1] in ["군", "구", "도", "시"]:
        trop_2016_san_TOT_QTY.iloc[i,1] = "국내"
    else:
        trop_2016_san_TOT_QTY.iloc[i,1] = "수입"
        
# 품목별 국내, 수입산 거래량
grouped_2016_trop_san_TOT_QTY = pd.DataFrame(trop_2016_san_TOT_QTY.groupby(["PUM_NM", "SAN_NM"])["TOT_QTY"].sum()).reset_index()

# 정규화
for i in range(0, 7):
    sum = (grouped_2016_trop_san_TOT_QTY.iloc[2*i, 2] + grouped_2016_trop_san_TOT_QTY.iloc[2*i+1, 2])
    grouped_2016_trop_san_TOT_QTY.iloc[2*i, 2] /= sum
    grouped_2016_trop_san_TOT_QTY.iloc[2*i+1, 2] /= sum
    
# 2019년 열대과일의 생산지 추출
trop_2019_san_TOT_QTY = df_2019[df_2019["PUM_NM"].isin(tropical_fruits)]
trop_2019_san_TOT_QTY = pd.DataFrame(trop_2019_san_TOT_QTY.groupby(["PUM_NM", "SAN_NM"])["TOT_QTY"].sum()).reset_index()

# 2019년 열대과일의 생산지를 국내와 수입으로 분류
for i in range(568):
    if trop_2019_san_TOT_QTY.iloc[i,1][-1] in ["군", "구", "도", "시"]:
        trop_2019_san_TOT_QTY.iloc[i,1] = "국내"
    else:
        trop_2019_san_TOT_QTY.iloc[i,1] = "수입"
        
# 품목별 국내, 수입산 거래량
grouped_2019_trop_san_TOT_QTY = pd.DataFrame(trop_2019_san_TOT_QTY.groupby(["PUM_NM", "SAN_NM"])["TOT_QTY"].sum()).reset_index()

# 정규화
for i in range(0, 7):
    sum = (grouped_2019_trop_san_TOT_QTY.iloc[2*i, 2] + grouped_2019_trop_san_TOT_QTY.iloc[2*i+1, 2])
    grouped_2019_trop_san_TOT_QTY.iloc[2*i, 2] /= sum
    grouped_2019_trop_san_TOT_QTY.iloc[2*i+1, 2] /= sum
    
# save pickle
with open("./pickles/hypo5_grouped_2016_trop_san_TOT_QTY_data.pickle", "wb") as fw:
    pickle.dump(grouped_2016_trop_san_TOT_QTY, fw)
with open("./pickles/hypo5_grouped_2019_trop_san_TOT_QTY_data.pickle", "wb") as fw:
    pickle.dump(grouped_2019_trop_san_TOT_QTY, fw)

In [35]:
# load pickle
with open("./pickles/hypo5_grouped_2016_trop_san_TOT_QTY_data.pickle", "rb") as fr:
    grouped_2016_trop_san_TOT_QTY = pickle.load(fr)
with open("./pickles/hypo5_grouped_2019_trop_san_TOT_QTY_data.pickle", "rb") as fr:
    grouped_2019_trop_san_TOT_QTY = pickle.load(fr)

# 품목_연도
for i in range(14):
    grouped_2016_trop_san_TOT_QTY.iloc[i, 0] = grouped_2016_trop_san_TOT_QTY.iloc[i, 0] + "_2016"
for i in range(14):
    grouped_2019_trop_san_TOT_QTY.iloc[i, 0] = grouped_2019_trop_san_TOT_QTY.iloc[i, 0] + "_2019"

# 2016년과 2019년을 동시에 표현
grouped_2016_2019_trop_san_TOT_QTY = pd.concat([grouped_2016_trop_san_TOT_QTY,
                                                grouped_2019_trop_san_TOT_QTY]).sort_values("PUM_NM")

# 소수점 4자리까지 표현
grouped_2016_2019_trop_san_TOT_QTY['TOT_QTY'] = grouped_2016_2019_trop_san_TOT_QTY['TOT_QTY'].round(4)

# 시각화 - 2016년과 2019년의 열대과일 국내, 수입산 거래량 비율
fig = px.bar(grouped_2016_2019_trop_san_TOT_QTY, x = 'PUM_NM', y = "TOT_QTY", color = 'SAN_NM', text_auto = True)
fig.update_layout(title = "2016년과 2019년의 열대과일 7종의 국내, 수입산 거래량 비율", width = 1000, height = 600)
fig.show()

* 세로 2줄씩 2016년과 2019년의 품목별 국내산, 수입산 거래량 비율을 나타내었다.
* 기술 발전으로 인해 시간이 지나면서 열대과일의 국내 생산량이 늘어났을 것이라고 예측하여 가설을 세웠지만,
* 2016년 대비 2019년에 열대과일의 국내 생산량 비율이 더 높아진 품목은 7개 품목 중 람부탄 하나 뿐이었다.
* 나머지 6종은 국내 생산량 비율이 소폭 감소했다.
* "일반적으로 2016년과 2019년 사이에 열대과일의 국내산 거래량이 수입산 거래량보다 적어졌다" 라고 단정짓기에는 통계적 검정 과정이 필요할 것이지만, 본 데이터 분석 결과로는 2016년과 2019년 사이에 열대과일의 국내산 거래량이 수입산 거래량보다 적어졌다고 판단했다.  
* **시각화를 해 본 후**
  * 3년이 지난 후 열대과일의 수입산 대비 국내산 거래량 비율이 줄어든 이유에 대해서는 외부 데이터를 이용하여 추가적인 분석이 필요해 보인다.